In [41]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [35]:
print('reading train and test data')
#id and 57 predictor columns such as number of bedrooms and stufff like that
properties = pd.read_csv('data/properties_2016.csv')

#train has the logerror, a parcelid and a transaction date
train = pd.read_csv('data/train_2016_v2.csv')

#read in the test_ids needed for submission
test_ids = pd.read_csv('data/sample_submission.csv')

reading train and test data


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
#fill median for numeric columns
for c in properties.dtypes[properties.dtypes == 'float64'].index.values:
    properties[c].fillna(properties[c].median(skipna=True), inplace = True)
    
#fill -1 for categorical columns
for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == object:
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))

In [45]:
#merge the training data with the properties
train_df = train.merge(properties, how='left', on='parcelid')

#drop the id from the test dataframe
x_test = properties.drop(['parcelid'], axis=1)

#drop id, data and logerror(response) from the df to make the training df
x_train = train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)

#retrieve just the logerror of the training set
y_train = train_df["logerror"].values.astype(np.float32)

In [46]:
#######
# turn the general pandas dataframes into numpy arrays for use in the random forest regressions
#######
x_train = x_train.values.astype(np.float32, copy=False)
x_test = x_test.values.astype(np.float32, copy=False)
z_rf = RandomForestRegressor(max_depth= 8, n_estimators = 100)
# z_rf = LinearRegression()

z_rf.fit(x_train, y_train)

rf_y_test = z_rf.predict(x_test)

57

In [59]:
temp = list(zip(train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1).columns, z_rf.feature_importances_))
sorted(temp, key=lambda x: x[1], reverse=True)

[('structuretaxvaluedollarcnt', 0.11702571582213393),
 ('taxamount', 0.08936513623159062),
 ('taxvaluedollarcnt', 0.08679662240983137),
 ('finishedsquarefeet12', 0.08361748820772004),
 ('lotsizesquarefeet', 0.07215855247336864),
 ('longitude', 0.0604218990309166),
 ('latitude', 0.05890929486567658),
 ('calculatedfinishedsquarefeet', 0.0543917378848465),
 ('yearbuilt', 0.05158051296576212),
 ('landtaxvaluedollarcnt', 0.04366744792248092),
 ('regionidzip', 0.04187563240604183),
 ('rawcensustractandblock', 0.022534523382400487),
 ('propertyzoningdesc', 0.01856541886978537),
 ('regionidcity', 0.017914108223547026),
 ('bedroomcnt', 0.016582804648357576),
 ('censustractandblock', 0.015373892583787248),
 ('bathroomcnt', 0.014629823018366439),
 ('regionidneighborhood', 0.01217653133842616),
 ('buildingqualitytypeid', 0.01155659239886758),
 ('propertycountylandusecode', 0.011279140152263403),
 ('propertylandusetypeid', 0.009862826646055034),
 ('fullbathcnt', 0.008853813213945209),
 ('finishedfl

In [ ]:
"""
#uncomment this section and run locally in order to optimize the paramaters, you can add others as well.
from sklearn.model_selection import GridSearchCV
#set the paramater dict to try
param_grid = {
                 'n_estimators': [500, 1000, 1500, 2000],
                 'max_depth': [5, 7, 9]
             }
             
grid = GridSearchCV(z_rf, param_grid, cv=10)

grid.fit(x_train, y_train)

print(grid.best_score_)
print(grid.best_params_)
"""

In [47]:
test_columns = ['201610','201611','201612','201710','201711','201712']

for i in test_columns:
    test_ids[i] = [float(format(house, '.4f')) for house in rf_y_test]
    
test_ids.to_csv('cam_rf_model_component.csv', index=False)